In [1]:
import pandas as pd
import xgboost as xgb
import datetime
import os
import subprocess
import sys
import json

先にGoogle Cloud Storageに学習データをアップロードしておく。

In [2]:
# バケット名
BUCKET_NAME = 'itoc-232101-mlengine'


データをダウンロードする

In [3]:
data_filename = 'data.csv'
target_filename = 'target.csv'
data_dir = 'gs://' + BUCKET_NAME

subprocess.check_call(['gsutil', 'cp', os.path.join(data_dir, data_filename), data_filename])
subprocess.check_call(['gsutil', 'cp', os.path.join(data_dir, target_filename), target_filename])

0

データは今回XGBoostで学習をするために、One-Hotエンコーディング処理を施しておく。

In [4]:
pd.read_csv(data_filename, index_col=0).head()

,x1:種類_中古マンション等,x2:種類_宅地(土地),x3:種類_宅地(土地と建物),x4:種類_林地,x5:種類_農地,x6:地域_住宅地,x7:地域_商業地,x8:地域_宅地見込地,x9:地域_工業地,x10:坪単価_float,...,x74:取引時点_平成30年第２四半期,x75:取引時点_平成30年第３四半期,x76:改装_改装済,x77:改装_未改装,x78:取引の事情等_その他事情有り,x79:取引の事情等_私道を含む取引,x80:取引の事情等_調停・競売等,x81:取引の事情等_関係者間取引,x82:取引の事情等_隣地の購入,x83:取引の事情等_隣地の購入、関係者間取引
0,0,1,0,0,0,1,0,0,0,79000.000000,...,0,0,0,1,0,0,1,0,0,0
1,0,0,0,1,0,1,0,0,0,90662.852984,...,1,0,0,1,0,0,1,0,0,0
2,0,0,1,0,0,1,0,0,0,90662.852984,...,0,0,0,1,0,0,1,0,0,0
3,0,1,0,0,0,1,0,0,0,86000.000000,...,1,0,0,1,0,0,1,0,0,0
4,0,0,1,0,0,1,0,0,0,90662.852984,...,1,0,0,1,0,0,1,0,0,0


XGBoostへ渡すように配列の形式に戻す

In [5]:
data = pd.read_csv(data_filename, index_col=0).values
target = pd.read_csv(target_filename, index_col=0).values
target = target.reshape((target.size,))


In [6]:
target

array([ 5200000,    25000, 25000000, ...,  2300000,  4000000,  3100000])

データをDMatrixオブジェクトとして読み込む

In [7]:
dtrain = xgb.DMatrix(data, label=target)



XGBoostのモデルを学習させる

In [8]:
bst = xgb.train({}, dtrain, 20)

[19:45:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 64 extra nodes, 0 pruned nodes, max_depth=6
[19:45:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=6
[19:45:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=6
[19:45:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=6
[19:45:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=6
[19:45:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=6
[19:45:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=6
[19:45:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=6
[19:45:43] /work

モデルをファイルとして出力する

In [9]:
model_filename = 'model.bst'
bst.save_model(model_filename)

モデルファイルをGoogle Cloud Storageに

In [10]:
gcs_model_path = os.path.join('gs://', BUCKET_NAME,
    datetime.datetime.now().strftime('%Y%m%d_%H%M%S'), model_filename)
subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path])

0

# モデル作成

上記のコード部分を train.py というファイルにして保存し、次のようなディレクトリ構造を作成する。

```
xgboost_trainer/
    __init__.py(空ファイル)
    train.py
```

次の環境変数を設定し次のコマンドをコンソール上で実行することでローカルでモデルの学習を実行できる

```
TRAINING_PACKAGE_PATH="./xgboost_trainer/"
MAIN_TRAINER_MODULE="xgboost_trainer.train"
```

`> gcloud ml-engine local train --package-path $TRAINING_PACKAGE_PATH --module-name $MAIN_TRAINER_MODULE`

今度は次の環境変数を設定する

```
 PROJECT_ID=[PROJECT-ID]
 BUCKET_ID=[BUCKET-ID]
 JOB_NAME="xgboost_$(date +"%Y%m%d_%H%M%S")"
 JOB_DIR=gs://$BUCKET_ID/xgboost_job_dir
 TRAINING_PACKAGE_PATH="./xgboost_trainer/"
 MAIN_TRAINER_MODULE="xgboost_trainer.train"
 REGION=us-central1
 RUNTIME_VERSION=1.12
 PYTHON_VERSION=3.5
 SCALE_TIER=BASIC
```

次のコマンドを実行することで、クラウド上でトレーニングを行う

```
gcloud ml-engine jobs submit training $JOB_NAME \
  --job-dir $JOB_DIR \
  --package-path $TRAINING_PACKAGE_PATH \
  --module-name $MAIN_TRAINER_MODULE \
  --region $REGION \
  --runtime-version=$RUNTIME_VERSION \
  --python-version=$PYTHON_VERSION \
  --scale-tier $SCALE_TIER
```

クラウドに登録したジョブは次のコマンドで状態を確認できる

`> gcloud ml-engine jobs stream-logs $JOB_NAME`

# 予測

予測ファイルは`input.json`として次のように保存する

```input.json
[0,0,1,0,0,1,0,0,0,90662.8529839884,0,0,1,0,27453.2518195051,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,5.3,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0]
[0,1,0,0,0,1,0,0,0,93000.0,0,0,1,0,28000.0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4.0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0]
```

```
MODEL_DIR=gs://$BUCKET_ID/
VERSION_NAME="sample"
MODEL_NAME="itoc"
INPUT_FILE="input.json"
FRAMEWORK="XGBOOST"
```

次のようなコマンドでローカルで予測をすることができる
`> gcloud ml-engine local predict --model-dir=$MODEL_DIR --json-instances $INPUT_FILE --framework $FRAMEWORK`

次のコマンドでモデルにバージョンを指定してデプロイできる

```
> gcloud ml-engine versions create $VERSION_NAME \
      --model $MODEL_NAME --origin $MODEL_DIR \
      --runtime-version=1.12 --framework $FRAMEWORK \
      --python-version=3.5
```

デプロイしたバージョンを指定して予測を行う

`gcloud ml-engine predict --model $MODEL_NAME --version $VERSION_NAME --json-instances $INPUT_FILE`